# Library importing

In [1]:
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# data pre-proccessing

## data reading

In [2]:
chunks = pd.read_csv("/kaggle/input/data.csv", chunksize=5_000_000)
final_data = pd.concat(chunks, ignore_index=True)

In [3]:
del chunks

In [4]:
preprocessor=joblib.load("/kaggle/input/pre-proccess.pkl")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info 

## Data spliting

In [5]:
y=final_data['sales']
x=final_data.drop('sales',axis = 1)

# Model

In [6]:
del final_data
import gc
gc.collect()

0

In [7]:
categorical_cols = ['store_id', 'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 'state_id', 'item_category', 'item_subcategory']
numeric_cols = ["wm_yr_wk", "wday", "snap","year", "month", "day", "sell_price","lag_1",'price_flag','lag_7','snap_weekend','wday_x_snap','is_weekend']
for col in categorical_cols:
    if x[col].dtype == "object":
        x[col] = x[col].astype("category")
for col in numeric_cols:
    x[col] = pd.to_numeric(x[col], downcast="float")
del categorical_cols
del numeric_cols
gc.collect()

0

In [8]:
del col

In [9]:
gc.collect()

0

In [10]:
x=preprocessor.fit_transform(x)

In [11]:
model = XGBRegressor(
    objective='count:poisson',
    booster='gbtree',
    device='cuda',
    tree_method='hist',
    random_state=42,
    eval_metric='mae',
    learning_rate=0.05,
    max_depth=10,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=6,
    reg_alpha=2,
    n_estimators=3000,
    early_stopping_rounds=40,
    gamma=0.1
)

In [12]:
x_temp , x_test , y_temp , y_test = train_test_split(x , y , test_size = 0.20 , random_state = 42)
del x,y
gc.collect()
x_train , x_val , y_train , y_val = train_test_split(x_temp , y_temp , test_size = 0.15 , random_state = 42)
del x_temp,y_temp
gc.collect()

0

In [13]:
gc.collect()

0

In [14]:
model.fit(x_train,y_train,eval_set = [(x_val , y_val)])

[0]	validation_0-mae:1.51602
[1]	validation_0-mae:1.50378
[2]	validation_0-mae:1.48961
[3]	validation_0-mae:1.47695
[4]	validation_0-mae:1.46457
[5]	validation_0-mae:1.45274
[6]	validation_0-mae:1.44122
[7]	validation_0-mae:1.43038
[8]	validation_0-mae:1.41917
[9]	validation_0-mae:1.40812
[10]	validation_0-mae:1.39724
[11]	validation_0-mae:1.38653
[12]	validation_0-mae:1.37604
[13]	validation_0-mae:1.36559
[14]	validation_0-mae:1.35526
[15]	validation_0-mae:1.34500
[16]	validation_0-mae:1.33489
[17]	validation_0-mae:1.32797
[18]	validation_0-mae:1.31829
[19]	validation_0-mae:1.30899
[20]	validation_0-mae:1.29988
[21]	validation_0-mae:1.29090
[22]	validation_0-mae:1.28287
[23]	validation_0-mae:1.27415
[24]	validation_0-mae:1.26557
[25]	validation_0-mae:1.25711
[26]	validation_0-mae:1.24960
[27]	validation_0-mae:1.24128
[28]	validation_0-mae:1.23307
[29]	validation_0-mae:1.22502
[30]	validation_0-mae:1.21795
[31]	validation_0-mae:1.21031
[32]	validation_0-mae:1.20280
[33]	validation_0-ma

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=40,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=3000, n_jobs=None,
             num_parallel_tree=None, objective='count:poisson', ...)

In [15]:
model.score(x_train,y_train) , model.score(x_test,y_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [15:53:41] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


(0.7838983706365137, 0.7300637740995576)

In [16]:
joblib.dump(model,'/kaggle/working/xgboost_model.pkl')

['/kaggle/working/xgboost_model.pkl']